In [2]:
from pathlib import Path
from wildlife_datasets.datasets import SeaTurtleID2022
from wildlife_tools.data import ImageDataset
from wildlife_datasets.splits import ClosedSetSplit
import os

import torch.nn as nn
import torchvision.transforms as T

os.environ['KAGGLE_USERNAME'] = "nashadammuoz"
os.environ['KAGGLE_KEY'] = "KGAT_ef90b0c2a32630082bb12f03344b4421"

root = Path("dataset")

SeaTurtleID2022.get_data(root=root)
dataset_df = SeaTurtleID2022(root=root, category_name="head", img_load="bbox").df
dataset_df.head(5)

/Users/lichclass/Documents/GitHub/Federated_SeaTurtleReID/.venv/lib/python3.13/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


DATASET SeaTurtleID2022: DOWNLOADING STARTED.
You are trying to download an already downloaded dataset.
        This message may have happened to due interrupted download or extract.
        To force the download use the `force=True` keyword such as
        get_data(..., force=True) or download(..., force=True).
        


,image_id,identity,path,bbox,date,orientation,segmentation,original_split,clarity
0,1,t001,turtles-data/data/images/t001/CAluWEgwPX.JPG,"[644.0, 441.0, 70.0, 78.0]",2014-07-14,left,"{'counts': [858927, 7, 1322, 23, 1306, 34, 129...",test,3
1,2,t001,turtles-data/data/images/t001/EKyrFKHQzh.JPG,"[1228.0, 476.0, 110.0, 133.0]",2014-07-14,right,"{'counts': [1637406, 1, 1329, 6, 1326, 8, 1324...",test,2
2,3,t001,turtles-data/data/images/t001/ELAvEqeXxT.JPG,"[356.0, 368.0, 119.0, 157.0]",2014-07-14,left,"{'counts': [474978, 29, 1303, 32, 1301, 33, 12...",test,2
3,4,t001,turtles-data/data/images/t001/IxRLFwTGCv.JPG,"[1028.0, 500.0, 143.0, 122.0]",2010-07-02,topright,"{'counts': [1155982, 9, 1113, 14, 1109, 17, 11...",train,3
4,5,t001,turtles-data/data/images/t001/LKCJAhfLBJ.JPG,"[1128.0, 582.0, 50.0, 68.0]",2014-07-14,right,"{'counts': [1504206, 8, 1325, 20, 1313, 26, 13...",test,4


In [3]:
def safe_split(df, name, seed):
    if len(df) == 0:
        raise ValueError(f"DataFrame for {name} is empty, cannot split.")
        
    splitter = ClosedSetSplit(ratio_train=0.5, seed=seed)
    splits = splitter.split(df)
    
    if len(splits) == 0:
         print(f"WARNING: Splitter returned no splits for {name}")
         return df, df

    gallery_idx, query_idx = splits[0]
    
    # Verify indices are valid
    if gallery_idx.max() >= len(df) or query_idx.max() >= len(df):
         raise IndexError(f"Splitter returned invalid indices for {name}. Max idx: {gallery_idx.max()}, DF len: {len(df)}")

    gal_df = df.iloc[gallery_idx].reset_index(drop=True)
    qry_df = df.iloc[query_idx].reset_index(drop=True)
    return gal_df, qry_df

def generate_query_gallery_splits(df, seed, img_size, root):
    gallery_df, query_df = safe_split(df, "Dataset Split", seed)
    t_eval = T.Compose([
        T.Resize((img_size, img_size)),
        T.ToTensor(),
        T.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225]),
    ])
    gallery_set = ImageDataset(
        gallery_df,
        root=root,
        transform=t_eval,
        col_path='path',
        col_label='identity'
    )
    query_set = ImageDataset(
        query_df,
        root=root,
        transform=t_eval,
        col_path='path',
        col_label='identity'
    )
    return gallery_set, query_set

test_df = dataset_df[dataset_df['original_split'] == 'test'].reset_index(drop=True)
test_gallery_set, test_query_set = generate_query_gallery_splits(test_df, seed=42, img_size=384, root=root)

test_gallery_set[0]


(
    tensor([[[-0.0287,  0.0227,  0.0569,  ..., -0.9192, -0.9192, -0.9192],
         [-0.0458,  0.0398,  0.0741,  ..., -0.9192, -0.9192, -0.9192],
         [-0.0629,  0.0398,  0.0912,  ..., -0.9020, -0.9192, -0.9192],
         ...,
         [-0.9020, -0.8849, -0.8849,  ..., -0.4226, -0.4054, -0.4054],
         [-0.8849, -0.8678, -0.8335,  ..., -0.4568, -0.4226, -0.3883],
         [-0.9020, -0.8678, -0.8164,  ..., -0.4739, -0.4226, -0.3712]],

        [[ 0.7829,  0.8004,  0.7829,  ...,  0.3978,  0.3978,  0.3978],
         [ 0.7654,  0.8179,  0.8004,  ...,  0.3978,  0.3978,  0.3978],
         [ 0.7654,  0.8179,  0.8004,  ...,  0.4153,  0.3978,  0.3978],
         ...,
         [ 0.2227,  0.2402,  0.2577,  ...,  0.3978,  0.4153,  0.4153],
         [ 0.2402,  0.2577,  0.2927,  ...,  0.3627,  0.3978,  0.4328],
         [ 0.2227,  0.2577,  0.3102,  ...,  0.3452,  0.3978,  0.4503]],

        [[ 2.3786,  2.4134,  2.4308,  ...,  1.3502,  1.3502,  1.3502],
         [ 2.2914,  2.4308,  2.4483,  

In [4]:
from torchvision.models import convnext_base, ConvNeXt_Base_Weights
import torch
import torch.nn.functional as F
import matplotlib.pyplot as plt
from tqdm import tqdm


# Prediction
class ConvNeXtBackbone(nn.Module):
    """
    ConvNeXt Backbone for Feature Extraction.
    """
    def __init__(self, embedding_dim=512, dropout=0.2, pretrained=True):
        super().__init__()
        weights = ConvNeXt_Base_Weights.IMAGENET1K_V1 if pretrained else None
        model = convnext_base(weights=weights)
        
        # Remove original classifier
        in_features = model.classifier[2].in_features
        model.classifier[2] = nn.Identity()
        self.backbone = model
        self.dropout = nn.Dropout(dropout)
        self.proj = nn.Linear(in_features, embedding_dim)

        self.contrastive_head = nn.Sequential(
            nn.Linear(embedding_dim, embedding_dim),
            nn.ReLU(),
            nn.Linear(embedding_dim, embedding_dim)
        )

    def forward(self, x):
        feat = self.backbone(x)
        feat = self.dropout(feat)
        emb = self.proj(feat)
        norms = torch.norm(emb, p=2, dim=1, keepdim=True)
        emb = F.normalize(emb, dim=1)
        projected_emb = self.contrastive_head(emb)
        return emb, projected_emb, norms.flatten()

# Wrapper Model
class ReIDModel(nn.Module): 
    """
    ReID Model combining Backbone and Head.
    """
    def __init__(self, backbone, head):
        super().__init__()
        self.backbone = backbone
        self.head = head

    def forward(self, x, labels=None, prediction=False):
        emb, projected_emb, norms = self.backbone(x)
        if labels is not None:
            logits = self.head(emb, norms, labels)
            return logits, projected_emb
        if prediction:
            return emb
        return projected_emb

def predict(model, query_img, gallery_set, device='cpu'):
    model.eval()
    model.to(device)
    
    print("Predicting query embedding...")
    with torch.no_grad():
        query_img = query_img.to(device).unsqueeze(0)
        query_emb = model(query_img, prediction=True).cpu()
        print("Query embedding predicted.")
        

    gallery_loader = torch.utils.data.DataLoader(gallery_set, batch_size=32, shuffle=False)
    gallery_embeddings = []
    gallery_labels = []

    with torch.no_grad():
        for batch_imgs, batch_labels in tqdm(gallery_loader, desc="Processing gallery batches"):
            batch_imgs = batch_imgs.to(device)
            batch_embs = model(batch_imgs, prediction=True).cpu()
            gallery_embeddings.append(batch_embs.cpu())
            gallery_labels.append(batch_labels)
        
    gallery_embeddings = torch.cat(gallery_embeddings, dim=0)
        
    # Compute distances
    query_emb = query_emb.cpu()
    similarities = torch.mm(query_emb, gallery_embeddings.t())
    sorted_indices = torch.argsort(similarities, dim=1, descending=True)
        
    return sorted_indices.squeeze().tolist(), gallery_labels

# Load pretrained model
backbone = ConvNeXtBackbone(embedding_dim=512)
eval_model = ReIDModel(backbone=backbone, head=nn.Identity()).to('cpu')

fl_method = 'fedprox'
backbone_state = Path('pretrained_models') / f"{fl_method}.pth"
backbone.load_state_dict(torch.load(backbone_state, map_location='cpu'))

# NOTE GENERATE EMBEDDINGS FOR GALLERY IMAGES FIRST
sample_query_img, sample_query_label = test_query_set[0]
sorted_gallery_indices, gallery_labels = predict(eval_model, sample_query_img, test_gallery_set, device='cpu')
print("Sorted gallery indices (closest to farthest):")
print(sorted_gallery_indices[5])
print("Corresponding gallery labels:")
print(gallery_labels[5])

fig, ax = plt.subplots(1, 6, figsize=(15, 5))
ax[0].imshow(sample_query_img.permute(1, 2, 0))
ax[0].set_title(f"Query ID: {sample_query_label}")
ax[0].axis('off')

RuntimeError: Error(s) in loading state_dict for ConvNeXtBackbone:
	Missing key(s) in state_dict: "contrastive_head.0.weight", "contrastive_head.0.bias", "contrastive_head.2.weight", "contrastive_head.2.bias". 